In [1]:
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    sns.set()
    import pandas as pd
    import pandas_datareader.data as web
    import pickle 
    import requests
    import tensorflow as tf
    import os
    from collections import deque
    import random


In [2]:
name = 'Double Recurrent Q-learning agent'
image_path = 'D:/kenneth/agent/images/'
path = 'D:/kenneth/trading/'
run_date=open(path+'run_date.txt').read()
start=('1970-01-01')
end=('2019-12-31')
days=30

In [3]:
if not os.path.exists(image_path+name):
        os.makedirs(image_path+name) 
# with open('D:/kenneth/trading/sp500/sp500tickers.txt','r') as f:
#     tickers=[line.rstrip('\n') for line in f]
tickers = ['^GSPC']

In [4]:
class Model:
    def __init__(self, input_size, output_size, layer_size, learning_rate, name):
        with tf.variable_scope(name):
            self.X = tf.placeholder(tf.float32, (None, None, input_size))
            self.Y = tf.placeholder(tf.float32, (None, output_size))
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                cell = tf.nn.rnn_cell.LSTMCell(layer_size, state_is_tuple = False)
            self.hidden_layer = tf.placeholder(tf.float32, (None, 2 * layer_size))
            self.rnn,self.last_state = tf.nn.dynamic_rnn(inputs=self.X,cell=cell,
                                                    dtype=tf.float32,
                                                    initial_state=self.hidden_layer) 
            self.logits = tf.layers.dense(self.rnn[:,-1], output_size)
            self.cost = tf.reduce_sum(tf.square(self.Y - self.logits))
            self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        
class Agent:

    LEARNING_RATE = 0.003
    BATCH_SIZE = 32
    LAYER_SIZE = 256
    OUTPUT_SIZE = 3
    EPSILON = 0.5
    DECAY_RATE = 0.005
    MIN_EPSILON = 0.1
    GAMMA = 0.99
    MEMORIES = deque()
    COPY = 1000
    T_COPY = 0
    MEMORY_SIZE = 300
    
    def __init__(self, state_size, window_size, trend, skip):
        self.state_size = state_size
        self.window_size = window_size
        self.half_window = window_size // 2
        self.trend = trend
        self.skip = skip
        tf.reset_default_graph()
        self.INITIAL_FEATURES = np.zeros((4, self.state_size))
        self.model = Model(self.state_size, self.OUTPUT_SIZE, self.LAYER_SIZE, self.LEARNING_RATE,
                           'real_model')
        self.model_negative = Model(self.state_size, self.OUTPUT_SIZE, self.LAYER_SIZE, self.LEARNING_RATE,
                                   'negative_model')
        self.sess = tf.InteractiveSession()
        self.sess.run(tf.global_variables_initializer())
        self.trainable = tf.trainable_variables()
    
    def _assign(self, from_name, to_name):
        from_w = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=from_name)
        to_w = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=to_name)
        for i in range(len(from_w)):
            assign_op = to_w[i].assign(from_w[i])
            self.sess.run(assign_op)

    def _memorize(self, state, action, reward, new_state, dead, rnn_state):
        self.MEMORIES.append((state, action, reward, new_state, dead, rnn_state))
        if len(self.MEMORIES) > self.MEMORY_SIZE:
            self.MEMORIES.popleft()

    def _select_action(self, state):
        if np.random.rand() < self.EPSILON:
            action = np.random.randint(self.OUTPUT_SIZE)
        else:
            action = self.get_predicted_action([state])
        return action

    def _construct_memories(self, replay):
        states = np.array([a[0] for a in replay])
        new_states = np.array([a[3] for a in replay])
        init_values = np.array([a[-1] for a in replay])
        Q = self.sess.run(self.model.logits, feed_dict={self.model.X:states, 
                                                   self.model.hidden_layer:init_values})
        Q_new = self.sess.run(self.model.logits, feed_dict={self.model.X:new_states, 
                                                       self.model.hidden_layer:init_values})
        Q_new_negative = self.sess.run(self.model_negative.logits, 
                                  feed_dict={self.model_negative.X:new_states, 
                                             self.model_negative.hidden_layer:init_values})
        replay_size = len(replay)
        X = np.empty((replay_size, 4, self.state_size))
        Y = np.empty((replay_size, self.OUTPUT_SIZE))
        INIT_VAL = np.empty((replay_size, 2 * self.LAYER_SIZE))
        for i in range(replay_size):
            state_r, action_r, reward_r, new_state_r, dead_r, rnn_memory = replay[i]
            target = Q[i]
            target[action_r] = reward_r
            if not dead_r:
                target[action_r] += self.GAMMA * Q_new_negative[i, np.argmax(Q_new[i])]
            X[i] = state_r
            Y[i] = target
            INIT_VAL[i] = rnn_memory
        return X, Y, INIT_VAL
    
    def get_state(self, t):
        window_size = self.window_size + 1
        d = t - window_size + 1
        block = self.trend[d : t + 1] if d >= 0 else -d * [self.trend[0]] + self.trend[0 : t + 1]
        res = []
        for i in range(window_size - 1):
            res.append(block[i + 1] - block[i])
        return np.array(res)
    
    def buy(self, initial_money):
        starting_money = initial_money
        states_sell = []
        states_buy = []
        inventory = []
        state = self.get_state(0)
        init_value = np.zeros((1, 2 * self.LAYER_SIZE))
        for k in range(self.INITIAL_FEATURES.shape[0]):
            self.INITIAL_FEATURES[k,:] = state
        for t in range(0, len(self.trend) - 1, self.skip):
            action, last_state = self.sess.run([self.model.logits,self.model.last_state],
                                                feed_dict={self.model.X:[self.INITIAL_FEATURES],
                                                            self.model.hidden_layer:init_value})
            action, init_value = np.argmax(action[0]), last_state
            next_state = self.get_state(t + 1)
            
            if action == 1 and initial_money >= self.trend[t]:
                inventory.append(self.trend[t])
                initial_money -= self.trend[t]
                states_buy.append(t)
#                 print('day %d: buy 1 unit at price %f, total balance %f'% (t, self.trend[t], initial_money))
            
            elif action == 2 and len(inventory):
                bought_price = inventory.pop(0)
                initial_money += self.trend[t]
                states_sell.append(t)
                try:
                    invest = ((close[t] - bought_price) / bought_price) * 100
                except:
                    invest = 0
                print(
                    'day %d, sell 1 unit at price %f, investment %f %%, total balance %f,'
                    % (t, close[t], invest, initial_money)
                )
            
            new_state = np.append([self.get_state(t + 1)], self.INITIAL_FEATURES[:3, :], axis = 0)
            self.INITIAL_FEATURES = new_state
        invest = ((initial_money - starting_money) / starting_money) * 100
        total_gains = initial_money - starting_money
        return states_buy, states_sell, total_gains, invest
            
    
    def train(self, iterations, checkpoint, initial_money):
        for i in range(iterations):
            total_profit = 0
            inventory = []
            state = self.get_state(0)
            starting_money = initial_money
            init_value = np.zeros((1, 2 * self.LAYER_SIZE))
            for k in range(self.INITIAL_FEATURES.shape[0]):
                self.INITIAL_FEATURES[k,:] = state
            for t in range(0, len(self.trend) - 1, self.skip):
                if (self.T_COPY + 1) % self.COPY == 0:
                    self._assign('real_model', 'negative_model')
                
                if np.random.rand() < self.EPSILON:
                    action = np.random.randint(self.OUTPUT_SIZE)
                else:
                    action, last_state = self.sess.run([self.model.logits,
                                                  self.model.last_state],
                                                  feed_dict={self.model.X:[self.INITIAL_FEATURES],
                                                             self.model.hidden_layer:init_value})
                    action, init_value = np.argmax(action[0]), last_state
                    
                next_state = self.get_state(t + 1)
                
                if action == 1 and starting_money >= self.trend[t]:
                    inventory.append(self.trend[t])
                    starting_money -= self.trend[t]
                
                elif action == 2 and len(inventory) > 0:
                    bought_price = inventory.pop(0)
                    total_profit += self.trend[t] - bought_price
                    starting_money += self.trend[t]
                    
                invest = ((starting_money - initial_money) / initial_money)
                new_state = np.append([self.get_state(t + 1)], self.INITIAL_FEATURES[:3, :], axis = 0)
                
                self._memorize(self.INITIAL_FEATURES, action, invest, new_state, 
                               starting_money < initial_money, init_value[0])
                self.INITIAL_FEATURES = new_state
                batch_size = min(len(self.MEMORIES), self.BATCH_SIZE)
                replay = random.sample(self.MEMORIES, batch_size)
                X, Y, INIT_VAL = self._construct_memories(replay)
                
                cost, _ = self.sess.run([self.model.cost, self.model.optimizer], 
                                        feed_dict={self.model.X: X, self.model.Y:Y,
                                                  self.model.hidden_layer: INIT_VAL})
                self.T_COPY += 1
                self.EPSILON = self.MIN_EPSILON + (1.0 - self.MIN_EPSILON) * np.exp(-self.DECAY_RATE * i)
            if (i+1) % checkpoint == 0:
                pass
                print('epoch: %d, total rewards: %f.3, cost: %f, total money: %f'%(i + 1, total_profit, cost,
                                                                                  starting_money))

In [5]:
def plot_all(ticker):
    close = df['Close']
    fig = plt.figure(figsize = (15,5), facecolor='yellowgreen', dpi=100)
    plt.plot(close, color='k', lw=3., alpha=0.5)
    plt.plot(close, '^', markersize=10, color='g', label = 'buying signal', markevery = states_buy)
    plt.plot(close, 'v', markersize=10, color='r', label = 'selling signal', markevery = states_sell)
    plt.title(name+'\n'+'Ticker: '+ticker)
    plt.legend()
    plt.xticks(rotation=45)
    plt.gca().axes.get_yaxis().set_visible(False)
    fig.tight_layout()
    plt.savefig(image_path+name+'/'+ticker+'.png', facecolor='yellowgreen', dpi=100)
    #plt.show()
    plt.close()

In [6]:
def display_only(days):
    new_buy = []
    new_sell = []
    display_days = days
    total_days= df.shape[0]
    display_from = total_days-display_days
    for i in(states_buy):
        if i > display_from:
            adj = i- display_from
            new_buy.append(adj)
    for i in(states_sell):
        if i > display_from:
            adj = i- display_from
            new_sell.append(adj)
    
    return(new_buy, new_sell)

In [7]:
def plot_by_days(days, ticker):
    close = df['Close'][-days:]
    fig = plt.figure(figsize = (15,5), facecolor='yellowgreen', dpi=100)
    plt.plot(close, color='k', lw=3., alpha=0.5)
    plt.plot(close, '^', markersize=12, color='g',alpha=1.0,label = 'buying signal', markevery = new_buy)
    plt.plot(close, 'v', markersize=12, color='r',alpha=1.0, label = 'selling signal', markevery = new_sell)
    plt.title(name+'\n'+'Last '+str(days)+' Daily Trade Recommendations')
    plt.legend()
    plt.xticks(rotation=45)
    plt.gca().axes.get_yaxis().set_visible(False)
    fig.tight_layout()
    plt.savefig(image_path+name+'/'+ticker+'_'+str(days)+'.png', facecolor='yellowgreen', dpi=100)
    #plt.show()
    plt.close()

In [8]:
initial_money = 10000
window_size = 300
skip = 1
iterations = 10
checkpoint = 1
batch_size = 32


for ticker in tickers:
    df = pd.read_csv(path+'sp500/'+run_date+'/'+ticker+'.csv', index_col=0, parse_dates=True)
    df =df[start:end]
    print('Started ticker:', ticker)
    
    close = df.Close.values.tolist()
   
    agent = Agent(state_size = window_size, 
              window_size = window_size, 
              trend = close, 
              skip = skip)
    agent.train(iterations = iterations, checkpoint = checkpoint, initial_money = initial_money)

    states_buy, states_sell, total_gains, invest = agent.buy(initial_money = initial_money)
    plot_all(ticker)
#     new_buy, new_sell = display_only(days)
#     plot_by_days(days, ticker)


Started ticker: ^GSPC
epoch: 1, total rewards: 45390.200245.3, cost: 2.269433, total money: 16796.860401
epoch: 2, total rewards: 100830.768631.3, cost: 439.608032, total money: 28126.248856
epoch: 3, total rewards: 39776.647205.3, cost: 808.966797, total money: 34722.847401
epoch: 4, total rewards: 56539.746101.3, cost: 268.326355, total money: 48837.856209
epoch: 5, total rewards: 19859.210831.3, cost: 158.168839, total money: 14823.120987
epoch: 6, total rewards: 58642.572956.3, cost: 538.646973, total money: 68642.572956
epoch: 7, total rewards: 85888.977821.3, cost: 433.690308, total money: 92849.557899
epoch: 8, total rewards: 32159.375530.3, cost: 719.713989, total money: 1045.305950
epoch: 9, total rewards: 106232.715847.3, cost: 543.302734, total money: 33191.156033
epoch: 10, total rewards: 30102.398201.3, cost: 334.105774, total money: 22099.808357
day 3284, sell 1 unit at price 140.639999, investment 3.222018 %, total balance 10004.389999,
day 3460, sell 1 unit at price 165

day 4696, sell 1 unit at price 272.059998, investment 0.014708 %, total balance 10044.540070,
day 4704, sell 1 unit at price 262.549988, investment -0.076123 %, total balance 10044.340057,
day 4708, sell 1 unit at price 261.029999, investment 0.180381 %, total balance 10044.810059,
day 4713, sell 1 unit at price 259.179993, investment 0.812943 %, total balance 10046.900055,
day 4716, sell 1 unit at price 262.510010, investment 1.089809 %, total balance 9787.400085,
day 4717, sell 1 unit at price 261.519989, investment -0.308770 %, total balance 10048.920074,
day 4724, sell 1 unit at price 266.470001, investment 0.225677 %, total balance 10049.520081,
day 4731, sell 1 unit at price 270.160004, investment 0.498473 %, total balance 10050.860077,
day 4745, sell 1 unit at price 277.929993, investment -0.111414 %, total balance 10050.550079,
day 4749, sell 1 unit at price 276.410004, investment 0.330310 %, total balance 10051.460083,
day 4753, sell 1 unit at price 283.660004, investment 0.27

day 5291, sell 1 unit at price 327.750000, investment 3.033635 %, total balance 9756.510101,
day 5292, sell 1 unit at price 328.890015, investment 2.070018 %, total balance 10085.400116,
day 5301, sell 1 unit at price 331.750000, investment 1.757564 %, total balance 9761.080139,
day 5307, sell 1 unit at price 326.450012, investment -1.090736 %, total balance 9426.780151,
day 5310, sell 1 unit at price 315.440002, investment -4.383144 %, total balance 9099.310150,
day 5311, sell 1 unit at price 314.899994, investment -4.820919 %, total balance 9414.210144,
day 5312, sell 1 unit at price 311.489990, investment -3.236934 %, total balance 9725.700134,
day 5314, sell 1 unit at price 315.230011, investment -1.797504 %, total balance 10040.930145,
day 5323, sell 1 unit at price 334.779999, investment 1.383970 %, total balance 10045.500153,
day 5326, sell 1 unit at price 335.839996, investment -0.068441 %, total balance 10045.270142,
day 5328, sell 1 unit at price 343.929993, investment 0.8858

day 5851, sell 1 unit at price 442.339996, investment -0.503844 %, total balance 9717.820221,
day 5852, sell 1 unit at price 443.380005, investment 0.569331 %, total balance 10161.200226,
day 5862, sell 1 unit at price 449.829987, investment -1.005723 %, total balance 9700.300232,
day 5866, sell 1 unit at price 451.890015, investment -0.972974 %, total balance 9703.880249,
day 5868, sell 1 unit at price 448.880005, investment 0.127146 %, total balance 10152.760254,
day 5873, sell 1 unit at price 450.769989, investment 0.602580 %, total balance 10155.460236,
day 5902, sell 1 unit at price 442.799988, investment 0.076842 %, total balance 10155.800232,
day 5916, sell 1 unit at price 450.190002, investment 0.975688 %, total balance 9254.300232,
day 5922, sell 1 unit at price 444.709991, investment -1.925285 %, total balance 9699.010223,
day 5923, sell 1 unit at price 445.779999, investment -1.465486 %, total balance 10144.790222,
day 5950, sell 1 unit at price 446.029999, investment -0.899

day 6613, sell 1 unit at price 644.369995, investment 0.615212 %, total balance 10178.640167,
day 6640, sell 1 unit at price 633.500000, investment -1.353182 %, total balance 10169.950165,
day 6648, sell 1 unit at price 647.890015, investment 0.448064 %, total balance 10172.840179,
day 6661, sell 1 unit at price 645.440002, investment 1.124932 %, total balance 9535.250153,
day 6663, sell 1 unit at price 661.510010, investment 2.596273 %, total balance 9544.670135,
day 6667, sell 1 unit at price 668.909973, investment 2.579390 %, total balance 10213.580109,
day 6677, sell 1 unit at price 667.679993, investment -0.812602 %, total balance 9535.880096,
day 6678, sell 1 unit at price 672.559998, investment 0.049093 %, total balance 10208.440094,
day 6685, sell 1 unit at price 667.919983, investment -0.167403 %, total balance 10207.320099,
day 6689, sell 1 unit at price 661.960022, investment -0.015101 %, total balance 10207.220123,
day 6693, sell 1 unit at price 668.479980, investment 0.245

...

day 11128, sell 1 unit at price 1797.020020, investment 2.382638 %, total balance 14971.488922,
day 11136, sell 1 unit at price 1839.780029, investment -0.053238 %, total balance 13141.758942,
day 11140, sell 1 unit at price 1845.160034, investment 0.897336 %, total balance 13141.798981,
day 11148, sell 1 unit at price 1877.170044, investment 1.737017 %, total balance 7553.378937,
day 11149, sell 1 unit at price 1867.630005, investment 0.719411 %, total balance 9421.008942,
day 11150, sell 1 unit at price 1868.199951, investment 0.470569 %, total balance 11289.208893,
day 11152, sell 1 unit at price 1841.130005, investment -1.744043 %, total balance 13130.338898,
day 11153, sell 1 unit at price 1858.829956, investment -1.022879 %, total balance 14989.168854,
day 11157, sell 1 unit at price 1866.520020, investment 0.309012 %, total balance 13122.908844,
day 11158, sell 1 unit at price 1857.439941, investment -0.778311 %, total balance 14980.348785,
day 11164, sell 1 unit at price 1885.5

day 11509, sell 1 unit at price 2083.389893, investment -0.474374 %, total balance 7370.599030,
day 11511, sell 1 unit at price 2102.439941, investment 0.906136 %, total balance 9473.038971,
day 11512, sell 1 unit at price 2096.919922, investment -0.345028 %, total balance 11569.958893,
day 11518, sell 1 unit at price 1940.510010, investment -6.888447 %, total balance 9749.648956,
day 11524, sell 1 unit at price 1951.130005, investment -6.467728 %, total balance 5850.408966,
day 11525, sell 1 unit at price 1921.219971, investment 1.479498 %, total balance 7771.628937,
day 11534, sell 1 unit at price 1958.030029, investment 4.841484 %, total balance 5799.198883,
day 11537, sell 1 unit at price 1938.760010, investment -2.460180 %, total balance 3828.248932,
day 11540, sell 1 unit at price 1881.770020, investment -1.676200 %, total balance 3777.778961,
day 11544, sell 1 unit at price 1951.359985, investment 0.128280 %, total balance 3845.048981,
day 11548, sell 1 unit at price 2013.430054

day 11780, sell 1 unit at price 2127.810059, investment -2.567005 %, total balance 7640.978424,
day 11782, sell 1 unit at price 2127.020020, investment -1.937263 %, total balance 9767.998444,
day 11785, sell 1 unit at price 2139.159912, investment -1.872497 %, total balance 9781.388336,
day 11786, sell 1 unit at price 2139.120117, investment -2.151709 %, total balance 11920.508453,
day 11787, sell 1 unit at price 2139.760010, investment -1.904371 %, total balance 14060.268463,
day 11788, sell 1 unit at price 2163.120117, investment 1.757015 %, total balance 16223.388580,
day 11790, sell 1 unit at price 2164.689941, investment -0.573677 %, total balance 16210.898590,
day 11792, sell 1 unit at price 2159.929932, investment 0.644417 %, total balance 16224.728424,
day 11807, sell 1 unit at price 2139.600098, investment -0.979740 %, total balance 9780.418610,
day 11808, sell 1 unit at price 2144.290039, investment -0.895236 %, total balance 11924.708649,
day 11811, sell 1 unit at price 2151

day 12137, sell 1 unit at price 2619.550049, investment -2.804672 %, total balance 14274.179108,
day 12140, sell 1 unit at price 2698.629883, investment 0.632816 %, total balance 16972.808990,
day 12147, sell 1 unit at price 2779.600098, investment 2.897462 %, total balance 17051.079010,
day 12156, sell 1 unit at price 2786.570068, investment 2.142499 %, total balance 17109.528961,
day 12166, sell 1 unit at price 2588.260010, investment -2.096688 %, total balance 17054.099030,
day 12171, sell 1 unit at price 2581.879883, investment -0.887528 %, total balance 17030.978912,
day 12178, sell 1 unit at price 2642.189941, investment 1.110917 %, total balance 14403.138824,
day 12179, sell 1 unit at price 2663.989990, investment 0.267980 %, total balance 17067.128815,
day 12185, sell 1 unit at price 2670.139893, investment -1.421378 %, total balance 14335.498932,
day 12188, sell 1 unit at price 2639.399902, investment -1.995076 %, total balance 16974.898834,
day 12190, sell 1 unit at price 266

day 12534, sell 1 unit at price 2979.389893, investment 0.113908 %, total balance 16779.887604,
day 12535, sell 1 unit at price 3000.929932, investment 0.755432 %, total balance 19780.817535,
day 12538, sell 1 unit at price 2997.959961, investment -0.313559 %, total balance 19771.387604,
day 12540, sell 1 unit at price 3006.729980, investment 0.034269 %, total balance 19772.417633,
day 12546, sell 1 unit at price 2977.620117, investment -0.242892 %, total balance 19765.167633,
day 12556, sell 1 unit at price 2938.129883, investment -0.072107 %, total balance 10984.637360,
day 12559, sell 1 unit at price 2995.679932, investment 3.742535 %, total balance 11010.047272,
day 12560, sell 1 unit at price 2989.689941, investment 1.276416 %, total balance 13999.737213,
day 12564, sell 1 unit at price 2995.989990, investment 1.946378 %, total balance 13989.007233,
day 12565, sell 1 unit at price 3004.520020, investment 1.153094 %, total balance 16993.527252,
